## 1. Introduction

### a. Project's goal

### b. Project's constraints

## 2. Computer Vision

In this part we will analyze the structure of the Computer vision implementation for our project. For this we will start by enumerating the different external libraries that we used, then we will explain how we initialize our scene in our code, to finally detail the methods used to detect the obstacle, the robot and the destination.

### 2.1 External Libraries

In computer vision we used external libraries to help us in various tasks, we thus reference them here before introducing our implementation.

We used:
1. OpenCV-Contrib-Python which is a library that contains all opencv functions plus some funcitons added by contributors. OpenCV provided a rich set of functions that helped us process the images of our camera.
2. Numpy library for its easy data structures and mathematical operations.
3. Aruco library to handle the detection of our Aruco tags.
4. Scipy to handle transformation between rotation matrixs,quaternions and Euler angles.

NOTE: from now on the term "tag" or "marker" refers to ArUCo Markers

### 2.2 Vision module structure

The computer vision module is divided into several sub modules, shown on the diagram below:
vision.py is the main file containing the Map class definition, its attributes and functions to be called in the main and through other modules.

### 2.3 Scene initialization

For the scene, we chose to create a Map class, which will be in charge of handing every vision variable and functions as attributes and methods, as shown below (which only contains the class attributes, the methods will be shown further below):

In [ ]:
import cv2 as cv
from constants import *

class Map:
    def __init__(self):
        # Initialilze camera video capture
        self.capture = cv.VideoCapture(0)
        # Drop the first x frames
        for _ in range(FIRST_FRAME):
            self.capture.read()
        
        # Define class attributes
        self.success = True
        self.raw_frame = None
        self.frame = None
        self.robot = 3*[None]
        self.destination = 3*[None]
        self.map_corners = {}
        self.found_corners = False
        self.found_robot = False
        self.found_destination = False
        self.obstacles = []
        self.obstacles_lines = []
        self.target_lines = []
        self.pose_est = 3*[None]

- The Map class starts by initializing a capture for our camera to take pictures from, using opencv's method VideoCapture.
- Then it drops the first X frame to let the camera adapt its settings to the environment.
- It then initializes the rest of the class attributes:
1. success: False if it fails an image treatment
2. raw_frame: Raw captured frame of the camera
3. frame: Raw frame treated
4. robot: Contains the robot position in (x,y) and its orientation.
5. destination: Contains the destination's position in (x,y) and the corners of the detected aruco tag
6. map_corners: if the aruco tags in the corners are detected, this maps them to the pixel that we will use to crop the image
7. found_corners: is true if the aruco tags in the corners are detected
8. found_robot: is true if the robot tag is detected
9. found_destination: is true if the destination tag is detected
10. obstacles: list of obstacles, each defined as a list of points
11. possible_lines: list of lines of the computed visibility graph
12. target_lines: list of points that creates the computed shortest path
13. pose_est: position estimation of the kalman filter

### 2.4 Detection of obstacles, Thymio and goal point

## 3. Global Navigation

### a. Path planning

### b. Robot's actuation

## 4 Local Navigation

### a. Obstacle avoidance (Artificial Neural Network)

### b. Completion of the obstacle avoidance

## 5. Position Estimation: Kalman Filter

### a. Empirical tests

### b. Kalman filtering theory

## 6. Video Demonstrations

### a. Global example

### b. Specific cases

#### i. Obstacle avoidance

#### ii. Kidnapping

#### iii. Change in goal's position

#### iv. Hidden Camera